In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm.auto import tqdm
import urllib.request
%matplotlib inline

### Data

In [3]:
def read_data(QAP_INSTANCE_URL, verbose = False):
    qap_instance_file = urllib.request.urlopen(QAP_INSTANCE_URL)

    line = qap_instance_file.readline()
    n = int(line.decode()[:-1].split()[0])
    if verbose:
        print('Problem size: %d' % n)

    A = np.empty((n, n))
    if n != 50 and n != 60 and n != 80:
        qap_instance_file.readline()
    for i in range(n):
        line = qap_instance_file.readline()
        A[i, :] = list(map(int, line.decode()[:-1].split()))
    if verbose:
        print('Flow matrix:\n', A)
    
    B = np.empty((n, n))
    if n != 50:
        qap_instance_file.readline()
    
    for i in range(n):
        if n == 30:
            line1 = qap_instance_file.readline()
            line2 = qap_instance_file.readline()
            line = line1 + line2
        else:
            line = qap_instance_file.readline()
        #print(line, i)   
        B[i, :] = list(map(int, line.decode()[:-1].split()))
    if verbose:
        print('Distance matrix:\n', B)
    return A,B,n

In [4]:
def plot_costs(c_min, data, opt, name):
    plt.figure(figsize=(12,4))
    plt.title(f'Costs of {name}\nfound minimum: {c_min:.3f}, optimal:{opt:.3f}', fontdict = {'fontsize' : 20})
    plt.plot(data[:,0], label="Top scores")
    plt.plot(data[:,1], label="Mean scores")
    plt.plot(data[:,2], label="Worst scores")
    plt.xlabel("Iteration")
    plt.ylabel("Cost")
    plt.legend(loc='upper right')
    plt.show()

    
def plot_histogram(costs):
    plt.figure(figsize=(12,4))
    plt.xlim(7500, 9200)
    plt.hist(costs, bins=150)
    plt.title("Histogram of costs")
    plt.xlabel("Cost")
    plt.ylabel("Number of results")
    plt.show()

### SGA algorithm

In [50]:
'''def q3ap_objective_function(p, q):
    s = 0.0
    for i in range(n):
        s += (B[i, :] * B[i, :] * A[p[i], p] * A[q[i], q]).sum()
    return s'''
def q3ap_objective_function(p, q):
    s = 0.0
    for i in range(n):
        s += (A[i, :] * A[i, :] * B[p[i], p] * B[q[i], q]).sum()
    return s

In [51]:
def ls_mutation(p, q, K = 1):
    n = len(p)
    if K == 0:
        return p
    
    permutations = []
    for i in range(0, n):
        tmp = p.copy()
        tmp[0], tmp[i] = tmp[i], tmp[0]
        
        prmt = ls_mutation(tmp, q, K - 1)
            
        permutations.append((prmt, q3ap_objective_function(prmt, q)))
    
    return min(permutations, key = lambda k: k[1])[0] 

In [85]:
def lis_mutation(p, z, K = 1):
    curr_score = q3ap_objective_function(p, z)
    q = reverse_sequence_mutation(p, z)
    q = ls_mutation(p, z, K)
    mut_score = q3ap_objective_function(q, z)
    while(mut_score < curr_score):
        curr_score = mut_score
        p = q
        q = ls_mutation(p, z, K)
        mut_score = q3ap_objective_function(q, z)
    return p

In [86]:
def reverse_sequence_mutation(p , z):
    a = np.random.choice(len(p), 2, False)
    i, j = a.min(), a.max()
    q = p.copy()
    q[i:j+1] = q[i:j+1][::-1]
    return q

In [87]:
#Partialy Mapped Crossover
def PMX(ind1, ind2):
    a = np.random.choice(len(ind1), 2, False)
    i, j = a.min(), a.max()
    p1 = ind1[i:j+1]
    p2 = ind2[i:j+1]
    d_p1 = {p1[k] : p2[k] for k in range(len(p1))}
    d_p2 = {p2[k] : p1[k] for k in range(len(p2))}
    child1 = ind1.copy()
    child2 = ind2.copy()
    
    child1[i:j+1] = p2
    child2[i:j+1] = p1
    
    for k in range(i):
        while child1[k] in d_p2:
            child1[k] = d_p2[child1[k]]
        while child2[k] in d_p1:
            child2[k] = d_p1[child2[k]]
            
    for k in range(j+1, len(ind1)):
        s = 0
        while child1[k] in d_p2:
            child1[k] = d_p2[child1[k]]
            
        while child2[k] in d_p1:
            child2[k] = d_p1[child2[k]]
        
    return child2, child1

In [88]:
#Order Crossover
def OX(ind1, ind2):
    a = np.random.choice(len(ind1), 2, False)
    g_start, g_end = a.min(), a.max() # index of group split
    
    groups = []
    if g_start > 0:
        groups.append((0, g_start - 1))
    groups.append((g_start, g_end))
    if g_end < len(ind1) - 1:
        groups.append((g_end + 1, len(ind1) - 1))
        
    # rand index of group we want to inherit
    r = np.random.choice(len(groups))
    i, j = groups[r][0], groups[r][1]
    
    # fragments to inherit
    p1 = ind1[i:j+1]
    p2 = ind2[i:j+1]
    d_p1 = set(p1[k] for k in range(len(p1)))
    d_p2 = set(p2[k] for k in range(len(p2)))
    
    child1 = list(range(len(ind1)))
    child2 = list(range(len(ind2)))
    
    el_for_c1 = []
    el_for_c2 = []
    
    for k in range(j + 1, len(ind1)):
        if ind2[k] not in d_p1:
            el_for_c1.append(ind2[k])
        if ind1[k] not in d_p2:
            el_for_c2.append(ind1[k])
    
    for k in range(0, j + 1):
        if ind2[k] not in d_p1:
            el_for_c1.append(ind2[k])
        if ind1[k] not in d_p2:
            el_for_c2.append(ind1[k])
    
    child1[i:j+1] = p1
    child2[i:j+1] = p2
    
    idx = 0
    
    for k in range(j + 1, len(ind1)):
        child1[k] = el_for_c1[idx]
        child2[k] = el_for_c2[idx]
        idx += 1
    
    for k in range(0, i):
        child1[k] = el_for_c1[idx]
        child2[k] = el_for_c2[idx]
        idx += 1
        
    return child1, child2

In [89]:
def SGA(F1,F2,operator = PMX, number_of_iterations = 250, return_chromosome = False):
    population_size = 1000
    chromosome_length = n
    number_of_offspring = population_size
    crossover_probability = 0.95
    mutation_probability1 = 0.35
    mutation_probability2 = 0.25

    time0 = time.time()

    best_objective_value = np.Inf
    best_chromosome_p = np.zeros((1, chromosome_length))
    best_chromosome_q = np.zeros((1, chromosome_length))
    
    # generating an initial population
    current_population_p = np.zeros((population_size, chromosome_length), dtype=np.int64)
    for i in range(population_size):
        current_population_p[i, :] = np.random.permutation(chromosome_length)
        
    current_population_q = np.zeros((population_size, chromosome_length), dtype=np.int64)
    for i in range(population_size):
        current_population_q[i, :] = np.random.permutation(chromosome_length)

    # evaluating the objective function on the current population
    objective_values = np.zeros(population_size)
    for i in range(population_size):
        objective_values[i] = q3ap_objective_function(current_population_p[i, :], current_population_q[i, :])
    
    costs = np.zeros((number_of_iterations, 3)) # we will keep best, mean and worst cost in i-th iteration
    for t in tqdm(range(number_of_iterations)):
        
        # selecting the parent indices by the roulette wheel method
        fitness_values = objective_values.max() - objective_values
        if fitness_values.sum() > 0:
            fitness_values = fitness_values / fitness_values.sum()
        else:
            fitness_values = np.ones(population_size) / population_size
        parent_indices = np.random.choice(population_size, number_of_offspring, True, fitness_values).astype(np.int64)

        # creating the children population
        children_population_p = np.zeros((number_of_offspring, chromosome_length), dtype=np.int64)
        for i in range(int(number_of_offspring/2)):
            if np.random.random() < crossover_probability:
                children_population_p[2*i, :], children_population_p[2*i+1, :] = operator(current_population_p[parent_indices[2*i], :].copy(), current_population_p[parent_indices[2*i+1], :].copy())
            else:
                children_population_p[2*i, :], children_population_p[2*i+1, :] = current_population_p[parent_indices[2*i], :].copy(), current_population_p[parent_indices[2*i+1]].copy()
        if np.mod(number_of_offspring, 2) == 1:
            children_population_p[-1, :] = current_population_p[parent_indices[-1], :]
            
        children_population_q = np.zeros((number_of_offspring, chromosome_length), dtype=np.int64)
        for i in range(int(number_of_offspring/2)):
            if np.random.random() < crossover_probability:
                children_population_q[2*i, :], children_population_q[2*i+1, :] = operator(current_population_q[parent_indices[2*i], :].copy(), current_population_q[parent_indices[2*i+1], :].copy())
            else:
                children_population_q[2*i, :], children_population_q[2*i+1, :] = current_population_q[parent_indices[2*i], :].copy(), current_population_q[parent_indices[2*i+1]].copy()
        if np.mod(number_of_offspring, 2) == 1:
            children_population_q[-1, :] = current_population_q[parent_indices[-1], :]

        # mutating the children population
        for i in range(number_of_offspring):
            if np.random.random() < mutation_probability1:
                children_population_p[i, :] = F1(children_population_p[i, :], children_population_q[i, :])
                children_population_q[i, :] = F1(children_population_q[i, :], children_population_p[i, :])
            if np.random.random() < mutation_probability2:
                #children_population_p[i, :] = F1(children_population_p[i, :], children_population_q[i, :])
                #children_population_q[i, :] = F1(children_population_q[i, :], children_population_p[i, :])
                children_population_p[i, :] = F2(children_population_p[i, :], children_population_q[i, :])
                children_population_q[i, :] = F2(children_population_q[i, :], children_population_p[i, :])
                
            #if np.random.random() < mutation_probability1:
                
            #if np.random.random() < mutation_probability2:
                


        # evaluating the objective function on the children population
        children_objective_values = np.zeros(number_of_offspring)
        for i in range(number_of_offspring):
            children_objective_values[i] = q3ap_objective_function(children_population_p[i, :], children_population_q[i, :])

        # replacing the current population by (Mu + Lambda) Replacement
        objective_values = np.hstack([objective_values, children_objective_values])
        current_population_p = np.vstack([current_population_p, children_population_p])
        current_population_q = np.vstack([current_population_q, children_population_q])

        I = np.argsort(objective_values)
        current_population_p = current_population_p[I[:population_size], :]
        current_population_q = current_population_q[I[:population_size], :]
        objective_values = objective_values[I[:population_size]]
        
        # recording some statistics
        
        costs[t][0] = objective_values[0]
        costs[t][1] = objective_values.mean()
        costs[t][2] = objective_values[-1]
        
        if best_objective_value > objective_values[0]:
            best_objective_value = objective_values[0]
            best_chromosome_p = current_population_p[0, :]
            best_chromosome_q = current_population_q[0, :]
    
        #print(best_chromosome)
        print('%3d %14.8f %12.8f %12.8f %12.8f %12.8f' % (t, time.time() - time0, objective_values.min(), objective_values.mean(), objective_values.max(), objective_values.std()))
    #print("s_g_a time:", time.time() - time0)
    if return_chromosome:
        return best_objective_value, costs, best_chromosome_p, best_chromosome_q
    return best_objective_value, costs

In [84]:
A, B, n = read_data(f'http://anjos.mgi.polymtl.ca/anjos/qaplib/data.d/nug{12}.dat')
#SGA(ls_mutation, operator = PMX)
SGA(reverse_sequence_mutation, lis_mutation, operator = PMX)

  0     1.62548804 1666.00000000 4242.77200000 5620.00000000 999.42672969
  1     2.82012796 1666.00000000 3485.62000000 4682.00000000 795.07083936
  2     4.22590041 1540.00000000 2995.41400000 3932.00000000 570.95354330
  3     5.54823661 1540.00000000 2726.54000000 3454.00000000 431.02010672
  4     6.94593787 1540.00000000 2559.38000000 3118.00000000 350.78660123
  5     8.63573313 1334.00000000 2434.97200000 2894.00000000 309.08441439
  6    10.19886637 1334.00000000 2347.67800000 2746.00000000 277.14112707
  7    11.69951177 1334.00000000 2285.43600000 2660.00000000 265.44308977
  8    13.19178843 1236.00000000 2223.36800000 2564.00000000 252.08543111
  9    14.66436863 1236.00000000 2177.19400000 2510.00000000 243.25256497
 10    16.02137446 1236.00000000 2131.99800000 2442.00000000 231.97755925
 11    17.64477587 1236.00000000 2098.66400000 2388.00000000 218.31520127
 12    19.27011466 1236.00000000 2066.78200000 2344.00000000 213.49694254
 13    20.87239599 1236.00000000 2039.

KeyboardInterrupt: 